In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
import sys
import s3fs
import numpy as np
import fireducks.pandas as pd
import warnings
from dotenv import load_dotenv
from tqdm import tqdm
from pprint import pprint

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV
from xgboost import XGBClassifier
from gensim.models import Word2Vec, FastText

sys.path.append("../src")
from ml_utils import *

In [3]:
load_dotenv()
pd.set_option("display.max_columns", None)
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

# Data Preprocessing

In [4]:
with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])
df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)
df["clean_phrase_text"] = df["phrase_text"].apply(lambda x: nltk_text_preprocessing(x, True))
df = df[df["clean_phrase_text"] != ""]
df.head()

,phrase_text,sol,clean_phrase_text
0,* Aider à la mise en place de l évènement Shar...,0,aider mise place évènemer shareplan envoi rapp...
1,* Comprendre le métier des achats * Comment or...,0,comprendre métier achat comment organiser appe...
2,* Fendre du bois en forêt au merlin manuelleme...,0,fendre boi forêt merlin manuellemer débarder b...
4,"2 jours au CDI , 1 jour en arts plastiques , 1...",0,2 jour cdi 1 jour art plastique 1 jour musiqu ...
5,4 jours au sein du Bureau des affaires institu...,1,4 jour sein bureau affaire institutionnel fina...


# Model (static embedding (sparse or dense) + classification algorithm) pipeline

In [10]:
X = df["clean_phrase_text"]
y = df["sol"]

tokenized_texts = [text.split() for text in df["clean_phrase_text"]]
word2vec_model = Word2Vec(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)
fasttext_model = FastText(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)

static_embedding_models = {
    # Sparse embeddings
    "Bag of Words":CountVectorizer(),
    "TF":TfidfVectorizer(use_idf=False, norm = "l1"),
    "TF-IDF":TfidfVectorizer(),
    # Dense embeddings
    "Word2Vec": MeanEmbeddingVectorizer(model=word2vec_model),
    "FastText": MeanEmbeddingVectorizer(model=fasttext_model)
}

classification_models = {
    "Logistic Regression":LogisticRegression(),
    "Random Forest":RandomForestClassifier(),
    "Linear SVM":SVC(kernel="linear", probability=True),
    "Multinomial Naive Bayes":MultinomialNB(),
    "XGBoost":XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

dict_scores = {}

for embedding_name, embedding_model in tqdm(static_embedding_models.items(),
                                           desc="Static embeddings"):

    dict_scores[embedding_name] = {}
    
    for classification_name, classification_model in tqdm(classification_models.items(),
                                                         desc="Classification algorithms"):

        # Multinomial NB is not suited for dense vectors
        if embedding_name in ["Word2Vec", "FastText"] and classification_name == "Multinomial Naive Bayes":
            continue

        # For Logistic Regression and Linear SVM, and for dense embeddings, add standardisation
        if embedding_name in ["Word2Vec", "FastText"] and classification_name in ["Logistic Regression", "Linear SVM"]:

            pipeline = Pipeline(
                [
                    ("feature_extraction", embedding_model),
                    ("standardisation", StandardScaler()),
                    ("classifier", classification_model)
                ]
            )

        else:
        
            pipeline = Pipeline(
                [
                    ("feature_extraction", embedding_model),
                    ("classifier", classification_model)
                ]
            )

        scores = cross_val_score(pipeline, X, y, cv=10, scoring="accuracy")

        dict_scores[embedding_name][classification_name] = f"{np.mean(scores):.3f} ± {np.std(scores):.3f}"

Static embeddings: 100%|██████████| 5/5 [10:06<00:00, 121.30s/it]


In [12]:
pprint(dict_scores)

{'Bag of Words': {'Linear SVM': '0.609 ± 0.023',
                  'Logistic Regression': '0.655 ± 0.029',
                  'Multinomial Naive Bayes': '0.614 ± 0.052',
                  'Random Forest': '0.638 ± 0.033',
                  'XGBoost': '0.635 ± 0.023'},
 'FastText': {'Linear SVM': '0.607 ± 0.002',
              'Logistic Regression': '0.594 ± 0.028',
              'Random Forest': '0.595 ± 0.027',
              'XGBoost': '0.576 ± 0.042'},
 'TF': {'Linear SVM': '0.605 ± 0.012',
        'Logistic Regression': '0.604 ± 0.019',
        'Multinomial Naive Bayes': '0.606 ± 0.004',
        'Random Forest': '0.649 ± 0.018',
        'XGBoost': '0.651 ± 0.040'},
 'TF-IDF': {'Linear SVM': '0.629 ± 0.033',
            'Logistic Regression': '0.659 ± 0.032',
            'Multinomial Naive Bayes': '0.621 ± 0.024',
            'Random Forest': '0.646 ± 0.020',
            'XGBoost': '0.623 ± 0.036'},
 'Word2Vec': {'Linear SVM': '0.597 ± 0.034',
              'Logistic Regression': '0.5

Best model is TF-IDF + Logistic Regression

# Model tuning

In [ ]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("logreg", LogisticRegression(max_iter=1000))
])

params_grid = {
    "tfidf__max_df":[.7, .8, .9, 1],
    "tfidf__min_df":[.001, .01, .1],
    "tfidf__norm":["l1", "l2", None],
    "tfidf__sublinear_tf":[True, False],
    "tfidf__max_features":[10, 100, 1000, 10000],
    "tfidf__ngram_range":[]
    
}

grid_search = GridSearchCV(
    pipeline,
    params_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    verbose=0
)

grid_search.fit(X, y)